In [89]:
import sqlite3
import pandas as pd
import dhlab as dh
import sys
import math

In [163]:
db = "/mnt/disk1/Github/hyphens.db"
uni = "/mnt/disk2/NB-ngram-assoc/unigram-one-row.db"
hyp = "/mnt/disk1/Github/total.db"

In [3]:
def query(db, sql, param = ()):
    res = []
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        try:
            cur.execute(sql, param)
            res = cur.fetchall()
        except:
            print('en feil oppstod:', str(sys.exc_info()[1]) )
    return res

def pdquery(db, sql, params = ()):
    res = pd.DataFrame()
    with sqlite3.connect(db) as con:
        try:
            res = pd.read_sql_query(sql, con, params = params)
        except:
            print('en feil oppstod:', str(sys.exc_info()[1]) )
    return res

In [4]:
query(db, "select * from sqlite_master")

[('table',
  'hyp',
  'hyp',
  2,
  'CREATE TABLE hyp (freq int, first varchar, second varchar, full varchar)'),
 ('index', '_fw_', 'hyp', 1761977, 'CREATE INDEX _fw_ on hyp (full, freq)')]

In [68]:
pdquery(uni, "select * from sqlite_master").style

,type,name,tbl_name,rootpage,sql
0,table,unigram,unigram,2,"CREATE TABLE unigram (freq int, lang varchar, first varchar, json text)"
1,index,_lff_,unigram,1393414,"CREATE INDEX _lff_ on unigram(lang,first,freq)"
2,index,_lf_,unigram,1511081,"CREATE INDEX _lf_ on unigram(lang,freq)"
3,index,_flf_,unigram,1576805,"CREATE INDEX _flf_ on unigram (freq, lang, first)"


In [7]:
pdquery(db, "select * from hyp where full='Nasjonalbiblioteket'").sort_values(by="freq", ascending=False)

,freq,first,second,full
11,339,Nasjonal,biblioteket,Nasjonalbiblioteket
10,153,Nasjo,nalbiblioteket,Nasjonalbiblioteket
9,150,Nasjonalbiblio,teket,Nasjonalbiblioteket
8,79,Nasjonalbibliote,ket,Nasjonalbiblioteket
7,61,Nasjonalbib,lioteket,Nasjonalbiblioteket
6,52,Na,sjonalbiblioteket,Nasjonalbiblioteket
5,41,Nasjonalbi,blioteket,Nasjonalbiblioteket
4,9,Nasjonalbibli,oteket,Nasjonalbiblioteket
3,7,Nasjon,albiblioteket,Nasjonalbiblioteket
2,2,Nasjonalbibliotek,et,Nasjonalbiblioteket


In [164]:
pdquery(hyp, "select * from hyphendata where word='Nasjonalbiblioteket'").sort_values(by="freq", ascending=False)

,first,second,word,freq
11,Nasjonal,biblioteket,Nasjonalbiblioteket,2276
10,Nasjo,nalbiblioteket,Nasjonalbiblioteket,1816
9,Nasjonalbiblio,teket,Nasjonalbiblioteket,1372
8,Nasjonalbibliote,ket,Nasjonalbiblioteket,1357
7,Nasjonalbi,blioteket,Nasjonalbiblioteket,1301
6,Na,sjonalbiblioteket,Nasjonalbiblioteket,1108
5,Nasjonalbibli,oteket,Nasjonalbiblioteket,419
4,Nasjonalbib,lioteket,Nasjonalbiblioteket,398
3,Nas,jonalbiblioteket,Nasjonalbiblioteket,39
2,Nasjon,albiblioteket,Nasjonalbiblioteket,14


In [8]:
pdquery(db, "select * from hyp where full='pilspiss'").sort_values(by="freq", ascending=False)

,freq,first,second,full
1,66,pil,spiss,pilspiss
0,1,pils,piss,pilspiss


In [12]:
pdquery(db, "select * from hyp where full='bilpleie'").sort_values(by="freq", ascending=False)

,freq,first,second,full
0,2,bil,pleie,bilpleie


In [16]:
pdquery(db, "select * from hyp where full='barnehage'").sort_values(by="freq", ascending=False)

freq           3465
first         barne
second         hage
full      barnehage
Name: 3, dtype: object

In [213]:
def analyse(w):
    res = dict()
    for (f,s) in splits(w):
        fx = query(uni,f"select first, freq from unigram where lang = 'nob' and first = '{f}'" )
        sx = query(uni,f"select first, freq from unigram where lang = 'nob' and first = '{s}'" )
        try:
            fa = fx[0][1]
            sa = sx[0][1]
            #M = (fa + sa)/2
            #sd = math.sqrt( ((fa - M)**2 + (sa - M)**2)/2)
            #percent = abs(fa - sa)/sd
            res[(f,s, f"{f}{s}")] = (fa, sa, math.log(fa*sa), sa/fa)
        except:
            pass

    return (pd.DataFrame(res, index=['first_freq','second_freq','log_prod', 'sd'])
            .transpose()
            .sort_values(by='log_prod', ascending=False))

In [188]:
def compose(x, top=True):
    try:
        if top:
            res = pd.DataFrame(pdquery(hyp, f"select * from hyphendata where word='{x}'").sort_values(by="freq", ascending=False).iloc[0]).transpose()
        else:
            res = pd.DataFrame(pdquery(hyp, f"select * from hyphendata where word='{x}'").sort_values(by="freq", ascending=False))
    except:
        res = pd.DataFrame([(0,"","",x)], columns = "freq first second word".split())
    return res
                        

In [243]:
def splits(x):
    mains = [(x[0:i],x[i:]) for i in range(1,len(x))]
    #print(mains)
    fuge = []
    for prefix in mains:
        #print(prefix)
        if len(prefix[0]) > 1 and prefix[0].endswith('e') or prefix[0].endswith('s'):
            fuge.append((prefix[0][:-1], prefix[1]))
    return mains + fuge

In [214]:
pd.DataFrame(analyse("høysterett").sort_values(by='log_prod', ascending=False).iloc[0]).transpose()

,,,first_freq,second_freq,log_prod,sd
høyste,rett,høysterett,635.0,3557748.0,21.538263,5602.752756


In [256]:
analyse("barnehage").reset_index()

,level_0,level_1,level_2,first_freq,second_freq,log_prod,sd
0,barn,hage,barnhage,3986255.0,91538.0,26.622872,0.022963
1,barne,hage,barnehage,12212.0,91538.0,20.834684,7.495742
2,b,arnehage,barnehage,4089283.0,44.0,19.008070,0.000011
3,barnehag,e,barnehage,24.0,5847319.0,18.759548,243638.291667
4,bar,nehage,barnehage,920532.0,146.0,18.716314,0.000159
5,barneha,ge,barnehage,250.0,85136.0,16.873466,340.544000
6,barneh,age,barnehage,235.0,60378.0,16.467966,256.927660


In [261]:
words = "barnehage pilspiss tyttebær tyttebæret fredssamtaler klimakteri klimahysteri klimaavgifter klimamål klimapolitikk klimapolitikken miljøvern miljøarbeid verneombud dyrehage ombud hundehager tyttebær tilfreds tilfredshet flyvåpenet flyvåpen fly våpen".split()

In [224]:
analyse("formiddagslur")

,,,first_freq,second_freq,log_prod,sd
for,middagslur,formiddagslur,95899199.0,2614.0,26.247445,0.000027
formiddags,lur,formiddagslur,806.0,63449.0,17.750075,78.720844
formiddag,slur,formiddagslur,72854.0,152.0,16.220093,0.002086


In [252]:
analyse("høyesterettsjustitiarius")

,,,first_freq,second_freq,log_prod,sd
høyesterett,justitiarius,høyesterettjustitiarius,11512.0,7087.0,18.217163,0.615618
høyesteretts,justitiarius,høyesterettsjustitiarius,1267.0,7087.0,16.010425,5.593528


In [254]:
compose("høyesterettsjustitiarius")

,first,second,word,freq
17,høyeste,rettsjustitiarius,høyesterettsjustitiarius,3131


In [251]:
analyse("pilspiss")

first_freq  second_freq   log_prod        sd
pil  spiss pilspiss     65249.0      64263.0  22.156705  0.984889
     piss  pilpiss      65249.0       4174.0  19.422596  0.063970
pils piss  pilspiss     14097.0       4174.0  17.890347  0.296091

In [250]:
compose("formiddagslur")

,first,second,word,freq
2,formid,dagslur,formiddagslur,37


In [199]:
compose('pilspiss', top=False)

,first,second,word,freq
2,pil,spiss,pilspiss,173
1,pils,piss,pilspiss,8
0,pi,lspiss,pilspiss,1


In [258]:
%%time
pd.concat([pd.DataFrame(analyse(x).sort_values(by='log_prod', ascending=False).iloc[0]).transpose().reset_index() for x in words])

CPU times: user 177 ms, sys: 0 ns, total: 177 ms
Wall time: 215 ms


,level_0,level_1,level_2,first_freq,second_freq,log_prod,sd
0,barn,hage,barnhage,3986255.0,91538.0,26.622872,0.022963
0,barn,e,barne,3986255.0,5847319.0,30.779857,1.466870
0,pil,spiss,pilspiss,65249.0,64263.0,22.156705,0.984889
0,tytte,bær,tyttebær,1806.0,115329.0,19.154414,63.858804
0,tyttebær,et,tyttebæret,15235.0,50863444.0,27.376006,3338.591664
0,fred,samtaler,fredsamtaler,566174.0,186289.0,25.381711,0.329031
0,klim,akteri,klimakteri,60.0,23.0,7.229839,0.383333
0,klima,hysteri,klimahysteri,118227.0,12661.0,21.126643,0.107091
0,klima,avgifter,klimaavgifter,118227.0,120770.0,23.382005,1.021509
0,klima,mål,klimamål,118227.0,1505279.0,25.904851,12.732109


In [262]:
%%time
pd.concat([compose(x) for x in words])

CPU times: user 52.1 ms, sys: 0 ns, total: 52.1 ms
Wall time: 58.7 ms


,first,second,word,freq
7,barne,hage,barnehage,111244
2,pil,spiss,pilspiss,173
4,tytte,bær,tyttebær,6603
3,tytte,bæret,tyttebæret,42
4,freds,samtaler,fredssamtaler,908
0,klimakte,ri,klimakteri,1
3,klima,hysteri,klimahysteri,23
4,klimaav,gifter,klimaavgifter,49
3,kli,mamål,klimamål,552
6,klima,politikk,klimapolitikk,1834


In [170]:
pd.concat([compose(x) for x in "barnehage pilspiss fredssamtaler dyrehage hundehager".split()])

,first,second,word,freq
7,barne,hage,barnehage,111244
2,pil,spiss,pilspiss,173
4,freds,samtaler,fredssamtaler,908
2,dyre,hage,dyrehage,896
1,hunde,hager,hundehager,3
